# Softmax function

In [3]:
import tensorflow as tf

In [11]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3 , 4], [1, 7, 5, 5],
         [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

nb_classes = 3

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, nb_classes])

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
            
    # Testing & One-hot encoding
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9],
                                           [1, 3, 4, 3],
                                           [1, 1, 0, 1]]})
    print(all, "\n" ,sess.run(tf.argmax(all, 1)))

0 4.27797
200 0.459553
400 0.34653
600 0.253736
800 0.163117
1000 0.142677
1200 0.126879
1400 0.114198
1600 0.103793
1800 0.0951068
2000 0.0877477
[[  6.41164661e-05   9.99935746e-01   6.71450806e-08]
 [  8.42494369e-01   1.29315024e-02   1.44574121e-01]
 [  8.22199297e-07   2.04072848e-01   7.95926332e-01]] 
 [1 0 2]


# Softmax_cross_entropy_with_logits

In [12]:
x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3 , 4], [1, 7, 5, 5],
         [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

nb_classes = 3

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, nb_classes])

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# logit
logits = tf.matmul(X, W) + b

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
            
    # Testing & One-hot encoding
    all = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9],
                                           [1, 3, 4, 3],
                                           [1, 1, 0, 1]]})
    print(all, "\n" ,sess.run(tf.argmax(all, 1)))

0 6.49777
200 0.601614
400 0.463041
600 0.325732
800 0.181558
1000 0.147822
1200 0.131032
1400 0.117626
1600 0.106677
1800 0.0975727
2000 0.0898851
[[  3.16168560e-04   9.99683857e-01   4.23957864e-08]
 [  6.33922338e-01   2.56476458e-02   3.40429962e-01]
 [  4.71912301e-07   1.33799374e-01   8.66200149e-01]] 
 [1 0 2]


# Animal Classification

In [14]:
import numpy as np

In [41]:
# Predicting animal type based on various features

tf.set_random_seed(777)  # for reproducibility

xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

np_classes = 7 # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6, shape=(?, 1)

Y_one_hot = tf.one_hot(Y, nb_classes) # one hot shape=(?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # shape=(?, 7)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                labels=Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                X: x_data, Y: y_data
            })
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
              step, loss, acc))
            
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N, 1) = flatten => (N, ) matches pred.shape 
    # flatten : ex. [[1],[0]] => [1, 0]
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 5.142	Acc: 1.98%
Step:   100	Loss: 0.398	Acc: 88.12%
Step:   200	Loss: 0.255	Acc: 95.05%
Step:   300	Loss: 0.196	Acc: 95.05%
Step:   400	Loss: 0.161	Acc: 95.05%
Step:   500	Loss: 0.137	Acc: 97.03%
Step:   600	Loss: 0.120	Acc: 97.03%
Step:   700	Loss: 0.107	Acc: 98.02%
Step:   800	Loss: 0.097	Acc: 99.01%
Step:   900	Loss: 0.088	Acc: 99.01%
Step:  1000	Loss: 0.081	Acc: 100.00%
Step:  1100	Loss: 0.075	Acc: 100.00%
Step:  1200	Loss: 0.070	Acc: 100.00%
Step:  1300	Loss: 0.066	Acc: 100.00%
Step:  1400	Loss: 0.062	Acc: 100.00%
Step:  1500	Loss: 0.058	Acc: 100.00%
Step:  1600	Loss: 0.055	Acc: 100.00%
Step:  1700	Loss: 0.053	Acc: 100.00%
Step:  1800	Loss: 0.050	Acc: 100.00%
Step:  1900	Loss: 0.048	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 Tr